# Workshop Lecture 9, Exercise 1
For this exercise, you’ll be using macroeconomic data from the folder ../data/FRED.
1. There are seven decade-specific files named FRED_monthly_19X0.csv where X identifies the decade (X takes on the values 5, 6, 7, 8, 9, 0, 1). Write a loop that reads in all seven files as DataFrames and store them in a list.

Hint: Recall from the lecture that you should use pd.read_csv(. . . , parse_dates=[‘DATE’]) to automatically parse strings stored in the DATE column as dates.

2. Use pd.concat() to concate these data sets into a single DataFrame and set the DATE column as the index.
3. You realize that your data does not include GDP since this variable is only reported at quarterly frequency. Load the GDP data from the file GDP.csv and merge it with your monthly data using
an inner join.
4. You want to compute how (percent) changes of the variables in your data correlate with percent changes in GDP.
- Create a new DataFrame which contains the percent changes in CPI and GDP (using pct_change(), see also the last exercise in workshop 3), and the absolute changes for the remaining variables (using diff()).
- Compute the correlation of the percent changes in GDP with the (percent) changes of all other variables (using corr()). What does the sign and magnitude of the correlation coefficient tell you?

In [2]:
import pandas as pd

data = []

DATA_PATH = '/Users/lilapfageraas/Downloads/nhh/tech2/TECH2-H24/data/FRED'
for i in range(1, 8):
    file = pd.read_csv(f'{DATA_PATH}/FRED_monthly_{1940 + i*10}.csv', parse_dates=['DATE'])
    df = pd.DataFrame(file)
    data.append(df)


In [3]:
pd.concat(data, axis=0)

,DATE,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
0,1950-01-01,23.5,6.5,NaN,NaN,58.9
1,1950-02-01,23.6,6.4,NaN,NaN,58.9
2,1950-03-01,23.6,6.3,NaN,NaN,58.8
3,1950-04-01,23.6,5.8,NaN,NaN,59.2
4,1950-05-01,23.8,5.5,NaN,NaN,59.1
...,...,...,...,...,...,...
115,2019-08-01,256.0,3.6,2.1,0.6,63.1
116,2019-09-01,256.4,3.5,2.0,0.3,63.2
117,2019-10-01,257.2,3.6,1.8,-0.0,63.3
118,2019-11-01,257.9,3.6,1.6,-0.2,63.3


In [4]:
df = df.set_index('DATE')

In [5]:
df.head()

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,
2010-01-01,217.5,9.8,0.1,-0.8,64.8
2010-02-01,217.3,9.8,0.1,-1.1,64.9
2010-03-01,217.4,9.9,0.2,-1.6,64.9
2010-04-01,217.4,9.9,0.2,-0.4,65.2
2010-05-01,217.3,9.6,0.2,-0.5,64.9


In [6]:
file2 = pd.read_csv(f'{DATA_PATH}/GDP.csv', parse_dates=['DATE'])
df_GDP = pd.DataFrame(file2).set_index('DATE')
df_GDP

,GDP
DATE,
1947-01-01,2182.7
1947-04-01,2176.9
1947-07-01,2172.4
1947-10-01,2206.5
1948-01-01,2239.7
...,...
2023-04-01,22539.4
2023-07-01,22780.9
2023-10-01,22960.6


In [7]:
new_df = pd.merge(df, df_GDP, how='inner', on='DATE')
new_df.head()

,CPI,UNRATE,FEDFUNDS,REALRATE,LFPART,GDP
DATE,,,,,,
2010-01-01,217.5,9.8,0.1,-0.8,64.8,16582.7
2010-04-01,217.4,9.9,0.2,-0.4,65.2,16743.2
2010-07-01,217.6,9.4,0.2,-0.1,64.6,16872.3
2010-10-01,219.0,9.4,0.2,-1.0,64.4,16960.9
2011-01-01,221.2,9.1,0.2,-1.1,64.2,16920.6


In [18]:
pct_changes = new_df[['CPI', 'GDP']].pct_change()*100

variables = ['UNRATE', 'FEDFUNDS', 'REALRATE', 'LFPART']

pct_changes[variables] = df[variables].diff()

pct_changes.head()


,CPI,GDP,UNRATE,FEDFUNDS,REALRATE,LFPART
DATE,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-01,-0.045977,0.967876,0.0,0.0,1.2,0.3
2010-07-01,0.091996,0.771059,0.0,0.0,0.3,0.0
2010-10-01,0.643382,0.525121,-0.1,0.0,-0.9,-0.2
2011-01-01,1.004566,-0.237605,-0.2,0.0,-0.2,-0.1


In [19]:
pct_changes.corr().loc['GDP']

CPI        -0.161439
GDP         1.000000
UNRATE     -0.091552
FEDFUNDS   -0.183139
REALRATE    0.053237
LFPART     -0.193611
Name: GDP, dtype: float64